In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.to

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
!pip install datasets trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 19.4 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel

In [ ]:

# # Load the adapter a second time, with a different name, which will be our reference model.
# model.load_adapter("/path/to/peft", adapter_name="reference")

In [ ]:
import shutil
import torch
from accelerate import PartialState
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig
)
from trl import (
    PPOTrainer,
    ScriptArguments,
    PPOConfig,
    ModelConfig,
    get_peft_config,
    get_quantization_config,
    get_kbit_device_map,
)
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE

script_args = ScriptArguments(
    dataset_name="trl-internal-testing/tldr-preference-sft-trl-style",
    dataset_test_split="validation"
)

training_args = PPOConfig(
    output_dir="models/minimal/ppo_tldr",
    learning_rate=3e-6,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=64,
    total_episodes=30000,
    eval_strategy="steps",
    eval_steps=100,
    sft_model_path="bigscience/bloom-560m",
    reward_model_path="cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr",
    stop_token="eos",
    missing_eos_penalty=1.0,
)

model_args = ModelConfig(
    model_name_or_path="bigscience/bloom-560m"
)


shutil.rmtree(training_args.output_dir, ignore_errors=True)

# Load model & tokenizer
quantization_config = get_quantization_config(model_args)
torch_dtype = torch.float16

model_kwargs = dict(
    torch_dtype=torch_dtype,
    device_map=get_kbit_device_map() if quantization_config is not None else None,
    quantization_config=quantization_config,
)

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, padding_side="left")
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
if tokenizer.chat_template is None:
    tokenizer.chat_template = SIMPLE_CHAT_TEMPLATE

# Load models
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "dense"
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)



print("Loading reward and value models...")
value_model = AutoModelForSequenceClassification.from_pretrained(training_args.reward_model_path, num_labels=1)
reward_model = AutoModelForSequenceClassification.from_pretrained(training_args.reward_model_path, num_labels=1)

print("Loading policy model...")


# Load the base model.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # load_in_8bit=False,
    # llm_int8_threshold=6.0,
    # llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
# model = AutoModelForCausalLM.from_pretrained(
#     "mistralai/mixtral-8x7b-v0.1",
#     load_in_4bit=True,
#     quantization_config=bnb_config,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# model.config.use_cache = False

# Load the adapter.


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
)

# policy = AutoModelForCausalLM.from_pretrained(
#     training_args.sft_model_path,
#     device_map="auto",
#     quantization_config=bnb_config,
#     load_in_4bit=True,
#     # load_in_8bit =False,
#     # quantization_config=bnb_config,
#     # attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     # device_map="auto",
# )

bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # load_in_8bit=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
)

policy = AutoModelForCausalLM.from_pretrained(
    training_args.sft_model_path,
    device_map="cuda",
    # quantization_config=bnb_config,
    load_in_4bit=True,
    # load_in_8bit =False,
    # quantization_config=bnb_config,
    # attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    # device_map="auto",
)
# model = PeftModel.from_pretrained(
#     policy,
#     "/path/to/peft",
#     is_trainable=True,
#     adapter_name="train",
# )

model = get_peft_model(policy, lora_config)
model.print_trainable_parameters()


peft_config = get_peft_config(model_args)
# ref_policy = None if peft_config else AutoModelForCausalLM.from_pretrained(training_args.sft_model_path)
# #ref_policy = None
# policy = get_peft_model(policy, lora_config)
# ref_policy = get_peft_model(ref_policy, lora_config)

# Load dataset
dataset = load_dataset(script_args.dataset_name)
train_dataset = dataset["train"]
eval_dataset = dataset[script_args.dataset_test_split]

def prepare_dataset(dataset, tokenizer):
    def tokenize(element):
        input_ids = tokenizer.apply_chat_template(
            element["messages"][:1], padding=False, add_generation_prompt=True
        )
        return {"input_ids": input_ids, "lengths": len(input_ids)}

    dataset = dataset.map(tokenize, remove_columns=dataset.column_names, num_proc=2)
    dataset = dataset.filter(lambda x: x["lengths"] <= 512, num_proc=2)
    return dataset

with PartialState().local_main_process_first():
    train_dataset = prepare_dataset(train_dataset, tokenizer)
    eval_dataset = prepare_dataset(eval_dataset, tokenizer)

assert train_dataset[0]["input_ids"][-1] != tokenizer.eos_token_id, "EOS token found at end"

ref_model = AutoModelForCausalLM.from_pretrained(
    training_args.sft_model_path,
    device_map="auto",
    # quantization_config=bnb_config,
    load_in_4bit=True,
    # load_in_8bit =False,
    # quantization_config=bnb_config,
    # attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    # device_map="auto",
)

trainer = PPOTrainer(
    args=training_args,
    processing_class=tokenizer,
    model=policy,
    ref_model = None,
    # ref_model=ref_policy,
    reward_model=reward_model,
    value_model=reward_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
)
trainer.train()


# Save model
trainer.save_model(training_args.output_dir)

# Optional: push to hub
if training_args.push_to_hub:
    trainer.push_to_hub(dataset_name=script_args.dataset_name)

# Generate some completions for verification
trainer.generate_completions()


Loading tokenizer...
Loading reward and value models...
Loading policy model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Configure PEFT (e.g., LoRA)
peft_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Create trainer with PEFT
trainer = PPOTrainer(
    args=training_args,
    processing_class=tokenizer,
    model=model,
    ref_model=None,  # With PEFT, reference model can be None
    reward_model=reward_model,
    value_model=value_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
trainer = PPOTrainer(
    args=training_args,
    processing_class=tokenizer,
    model=policy,
    ref_model = ref_model,
    # ref_model=ref_policy,
    reward_model=reward_model,
    value_model=reward_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
)
trainer.train()

NameError: name 'peft_config' is not defined

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


===training policy===


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # load_in_8bit=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
)

policy = AutoModelForCausalLM.from_pretrained(
    training_args.sft_model_path,
    device_map="auto",
    # quantization_config=bnb_config,
    load_in_4bit=True,
    # load_in_8bit =False,
    # quantization_config=bnb_config,
    # attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    # device_map="auto",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]